<a href="https://colab.research.google.com/github/sviteribuben/ML_01/blob/main/jun_ml_linear_regression_II_les_1_%2B_HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В прошлом уроке мы узнали, как обучить и применить модель линейной регрессии, а так же как измерить качество модели. В этом уроке поговорим, как улучшить этот простой алгоритм и сделать обучение более эффективным. Мы познакомимся с понятиями "регуляризация" и "градиентный спуск".

Вы сможете обучать линейную регрессию, не "упираясь" в аппаратные ресурсы, а так же узнаете, как увеличить качество решения с помощью механизма регуляризации.

# Урок 1. Регуляризация

Регуляризация - это способ борьбы с таким явлением как "переобучение".

**Что такое переобучение?** Переобучение (англ. *overfitting*, буквально - "переподгонка") - негативное явление, возникающее, когда алгоритм обучения вырабатывает предсказания, которые слишком близко или точно соответствуют конкретному набору данных и поэтому не подходят для применения алгоритма к дополнительным данным или будущим наблюдениям.

Пример оверфиттинга проще всего привести в виде картинки. Пусть вы решаете задачу классификации на два класса ( о которой уже знаете с первых уроков этого курса)
* сначала модель, которая обучена не совсем хорошо (underfit) - разделяющая линия прямая и много "красных" точек неправильно классифицированы
* затем хорошо обученная модель (normal) - есть какой-то баланс
* переобученная модель (overfit) - разделяющая линия слишком сильно "облизывает" обучающие данные.
 
![overfit_example](https://248006.selcdn.ru/public/Data-science-3/img/overfit_example.png)

Чем же плохо переобучение? Когда в модель придут "новые" точки, то качество модели будет низким, потому что модель слишком сильно подогналась под обучающие данные. В этом случае говорят, что падает "обобщающая способность" - модель не получается обощить на новые данные.

**Почему происходит переобучение?** Всё дело в алгоритме подбора параметров модели. В модуле "Линейная регрессия. Часть I" в домашнем задании по уроку "Полиномиальная регрессия" нужно было подобрать такой параметр, как "степень полинома". Напомним общий алгоритм выбора *наилучшей степени полинома*:


1. возьмите все степени от 1 до 10 по порядку, без пропусков
1. для каждой степени полинома: обучите полиномиальную регрессию, сделайте предсказания и вычислите метрику качества (например, r2-score)
1. найдите степень полинома, при которой у модели будет лучший r2-score

Вспомним, что эта процедура называется *Grid Search* и помогает найти лучшие параметры для модели методом перебора.

Вот тут-то и поджидает нас переобучение! Мы выберем модель, которая даёт лучший скор на обучающих данных. А вот когда в модель придут новые данные, то качество может быть сильно хуже

**Как детектировать переобучение?** Перед нами стоит вопрос, как выбрать правильные пераметры модели, чтобы она не "оверфитнулась". Есть простой алгоритм, который называется "контроль на отложенной (валидационной) выборке". Тренировку модели из пункта (2) алгоритма *GridSearch* придётся немного усложнить:

* разбиваем обучающую выборку на две части: в одной части 80% обучающих примеров (эта часть называется *train set*), в другой части 20% обучающих примеров (эта часть называется *validation set*)
* выбираем метрику качества  модели (для регрессии, например, MSE)
* обучаем модель на тренировочном наборе данных
* делаем предсказания на валидационном наборе данных и вычисляем метрику качества

**Признак переобучения**: Если качество на валидации сильно хуже, чем качество на обучающем сете - всё плохо, модель переобучилась. Запомните этот признак, скоро мы его применим на практике. Такую модель нельзя использовать, с переобучением надо бороться!

**Как бороться с переобучением линейной регрессии?** Нам надо как-то "наказать" модель за то, что она слишком сильно подгоняется под обучающую выборку. Это можно сделать помощью регуляризации! Регуляризация - это специальная модификация модели линейной регрессии. В стандартной библиотеки sklearn есть два класса, в которых реализована регуляризация:
* sklearn.linear_model.Ridge, ссылка на доку https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
* sklearn.linear_model.Lasso, ссылка на доку https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

Чем отличаются эти два класса мы узнаем в следующих уроках этого модуля. Мы уже знакомы с классом `sklearn.linear_model.LinearRegression`, с который вообще не требует никаких параметров при создании, а вот классы `Ridge` и `Lasso` принимают на вход т.н. параметр регуляризации *alpha*, который принимает значения от $0$ до $1$ - чем ближе к единице, тем регуляризация сильнее, тем сильнее наказываем модель за сильную "подгонку" под обучающие данные.

В следующем уроке мы  продемонстрируем, как переобучается полиномиальная регрессия, а так же попытаемся победить этот эффект с помощью регуляризации

# Урок 2. Переобучение на примере линейной регрессии


Давайте проведём эксперимент и увидим на примере, как переобучается линейная регрессия. Для начала загрузим датасет из csv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import numpy as np
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML_01/3.10_non_linear.csv', sep=',')
data.head()

,x_train,y_train
0,0.138368,0.838812
1,0.157237,0.889313
2,0.188684,1.430040
3,0.685553,1.717309
4,0.874237,2.032588


Тренироваться будем на полиномиальной регрессии, потому что она сильно склонна к переобучению и мы уже знакомы с этой моделью. Сгенерируем полиномиальные признаки так же, как вы уже делали в модуле "Линейная регрессия. Часть I".

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error

def generate_degrees(source_data: list, degree: int):
    """Функция, которая принимает на вход одномерный массив, а возвращает n-мерный
    Для каждой степени от 1 до  degree возводим x в эту степень
    """
    return np.array([
          source_data**n for n in range(1, degree + 1)  
    ]).T

Обучаем модель на валидации, проверяем на контроле для степени полинома *degree=8*. Для разбиения мы воспользуемся функцией train_test_split (ссылка на доку: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Функция принимает следующие аргументы:

* $X$ и $y$ - массивы, которые хотим расщепить на валидацию и контроль
* *test_size* принимает значения от нуля до единицы и означает долю объектов, которые нужно отложить на валидацию (обычно выбирают значения между $0.15$ и $0.35$)
* *random_state* - любое целое число. Функция разбивает выборку случайным образом, но если вы задали параметр random_state, то разбиение не будет меняться в разных запусках программы. Это нужно для воспроизводимости исследований и является признаком хорошего тона в программировании.

In [40]:
degree = 8
X = generate_degrees(data['x_train'], degree)
y = data.y_train.values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)
model = Ridge(alpha=0).fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_train = model.predict(X_train)
print(f"Качество на валидации: {mean_squared_error(y_valid, y_pred).round(3)}")
print(f"Качество на обучении: {mean_squared_error(y_train, y_pred_train).round(3)}")

Качество на валидации: 0.119
Качество на обучении: 0.052


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=7.97797e-18): result may not be accurate.
  overwrite_a=True).T


Теперь обучим полиномиальную регрессию для степени *degree = 12* c параметром регуляризации *alpha=0*

In [50]:
degree = 12
X = generate_degrees(data['x_train'], degree)
y = data.y_train.values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)
model = Ridge(alpha=0).fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_train = model.predict(X_train)
print(f"Качество на валидации: {mean_squared_error(y_valid, y_pred).round(3)}")
print(f"Качество на обучении: {mean_squared_error(y_train, y_pred_train).round(3)}")

Качество на валидации: 0.125
Качество на обучении: 0.051


Как изменилась ошибка на обучении? Было $0.052$, стало $0.051$, т.е. *ошибка на обучении падает*.

Как изменилась ошибка на валидации? Было $0.119$, стало $0.125$, т.е. *ошибка на валидации растёт*.

Это же верный **признак переобучения**! Ошибка на валидации растёт, а на обучении падает. Степень полинома $n=12$ хуже, чем степень полинома $n=8$, модель переобучилась. Это печально, как же нам победить переобучение?

**Практическое задание**: предлагаю вам победить переобучение самостоятельно! Обучите *Ridge* регрессию с параметром регуляризации $\alpha=0.01$. 

Как изменилась ошибка на обучении? Как изменилась ошибка на валидации? Удалось ли победить переобучение?

In [51]:
# -- ВАШ КОД ТУТ --
degree = 8
X = generate_degrees(data['x_train'], degree)
y = data.y_train.values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)
model = Ridge(alpha=0.01, normalize=True).fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_train = model.predict(X_train)
print(f"Качество на валидации: {mean_squared_error(y_valid, y_pred).round(4)}")
print(f"Качество на обучении: {mean_squared_error(y_train, y_pred_train).round(4)}")

Качество на валидации: 0.0867
Качество на обучении: 0.1513


In [52]:
degree = 12
X = generate_degrees(data['x_train'], degree)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)
model = Ridge(alpha=0.01, normalize=True).fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_train = model.predict(X_train)
print(f"Качество на валидации: {mean_squared_error(y_valid, y_pred).round(4)}")
print(f"Качество на обучении: {mean_squared_error(y_train, y_pred_train).round(4)}")

Качество на валидации: 0.0847
Качество на обучении: 0.1456


если использовать нормализацию то все отрабатывает как надо! Как изменилась ошибка на обучении? Было  0.1513, стало  0.1456, т.е. ошибка на обучении падает.
Как изменилась ошибка на валидации? Было  0.0867, стало  0.0847, т.е. ошибка на валидации также снижается!
Переобучение **нивилировано** с изменением параметра $\alpha=0.01$ и "включением" нормализации! Ошибка на валидации падает, а на обучении падает алсо! 
Степень полинома  n=12  хуже, чем степень полинома  n=8 , модель годная! Пулим в прод)).

Я надеюсь, что переобучение с помощью регуляризации победить вам удалось!

Чему полезному мы научились?

* узнали, что такое переобучение
* научились детектировать его с помощью валидационной выборки
* смогли победить переобучение с помощью библиотечного класса Ridge